In [3]:
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib inline 

In [4]:
# to see all the columns when using head() or describe()
pd.set_option('display.max_columns',60)

In [7]:
df = pd.read_csv("diaries_trx_trunc_loans.csv", dtype={'account_startclose_balance': str})

In [14]:
df.head()

,Unnamed: 0,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,new_account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,...,trx_dq_round,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr,acc_unexplained_trx,150_days_or_more
0,2,KELDK21,NaN,20sep2012,22oct2013,397,13,04=Diaries Interview,02oct2013,10,2013,2013_10,05sep2012,56134804374600000,5.613480e+16,NaN,65134441430300000,NaN,Liability,Close,Informal,NaN,17oct2012,02oct2013,350,12,105138073330800000,65134441430300000,02oct2013,10,...,17.0,377,13,350,12,findev,"Borrowing, lending, savings or insurance media...",INFGRP,Informal group,3395,Borrowing from an informal Group,7. Closing Balance--End of last DQ,NaN,0.0,NaN,19.0,CLOSING BALANCE,NaN,NaN,NaN,NaN,0.0,0.000000,0,NaN,NaN,NaN,13.000000,0.0,0.0
1,27,KVIHK40,NaN,10sep2012,03oct2013,388,13,06=Cleaning interview,04oct2013,10,2013,2013_10,04sep2012,60137430710900000,6.013743e+16,NaN,60134547419200000,NaN,Liability,Close,Formal,NaN,27may2013,03oct2013,129,4,60138614254500000,60134547419200000,03oct2013,10,...,23.0,388,13,129,4,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,6. Closing Balance--End of last DQ,NaN,0.0,NaN,19.0,CLOSING BALANCE,NaN,NaN,NaN,NaN,101300.0,1191.765000,0,NaN,NaN,NaN,13.379310,0.0,0.0
2,70,KMAKE06,NaN,02oct2012,30sep2013,363,12,06=Cleaning interview,25sep2013,9,2013,2013_09,04sep2012,63136740549700000,6.313674e+16,NaN,63134425702500000,NaN,Liability,Start,Formal,NaN,22apr2013,24sep2013,155,5,63138432890300000,63134425702500000,22apr2013,4,...,21.0,202,6,0,0,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2762,Consumer/ personal loan (not payday loan),1. Starting balance (today),NaN,0.0,Increase,18.0,STARTING BALANCE,NaN,NaN,NaN,NaN,16800.0,197.647100,0,NaN,NaN,NaN,6.965517,0.0,0.0
3,96,KNBOK05,NaN,01nov2012,02nov2013,366,12,04=Diaries Interview,15mar2013,3,2013,2013_03,11sep2012,89136459180500000,8.913646e+16,NaN,64134429266300000,NaN,Liability,NaN,Informal,NaN,12feb2013,01nov2013,262,9,89136682215400000,64134429266300000,14mar2013,3,...,7.0,133,4,30,1,findev,"Borrowing, lending, savings or insurance media...",INFGRP,Informal group,3395,Borrowing from an informal Group,5. Interest accruing,NaN,0.0,Increase,1.0,Cash,NaN,NaN,NaN,NaN,600.0,7.058824,0,NaN,NaN,NaN,4.586207,0.0,0.0
4,162,KELDK37,NaN,24sep2012,30nov2013,432,14,04=Diaries Interview,08apr2013,4,2013,2013_04,04sep2012,105136540140100000,1.051365e+17,1.0,65134432186900000,NaN,Liability,Start,Formal,NaN,18mar2013,04oct2013,200,6,105136540174100000,65134432186900000,18mar2013,3,...,6.0,175,6,0,0,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,1. Starting balance (today),NaN,0.0,Increase,18.0,STARTING BALANCE,NaN,NaN,NaN,NaN,18000.0,211.764700,0,NaN,NaN,NaN,6.034483,0.0,0.0


In [25]:
df["obs_length"].sample(20)

549     372.0
866     224.0
917     219.0
473     369.0
1491    139.0
784     341.0
988     296.0
410     354.0
1525    242.0
1098    366.0
1607    423.0
1334    371.0
1694    200.0
1785    200.0
121     365.0
814     397.0
1258    357.0
356     372.0
1452    258.0
772     357.0
Name: obs_length, dtype: float64

### Insert column that checks whether we have at least 150 days of entries for the chunk of that account.

In [20]:
df.insert(df.shape[1],"obs_length",np.zeros(df.shape[0]))

In [24]:
for acc in df["new_account_ids"].unique():
    lna = df[df["new_account_ids"]==acc]
    days = lna["trx_stdtime_days_acc"].unique()
    l = days.max() - days.min()
    for index, row in df[df["new_account_ids"]==acc].iterrows():
        df.at[index,"obs_length"] = l